In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

In [ ]:
INFILE="../numu_ccqe_p00_p07_rad6_sig_bak_00430_00645.root"

reco_event_df = pd.DataFrame(rn.root2array(INFILE,treename='EventTree'))
reco_event_df = reco_event_df.drop_duplicates(subset=['run','subrun','event'])

reco_vtx3d_df = pd.DataFrame(rn.root2array(INFILE,treename='Vtx3DTree'))
reco_vtx3d_df = reco_vtx3d_df.drop_duplicates(subset=['run','subrun','event','vtx3d_id'])

reco_particle_df = pd.DataFrame(rn.root2array(INFILE,treename='ParticleTree'))
reco_particle_df = reco_particle_df.drop_duplicates(subset=['run','subrun','event','vtx3d_id','plane_id'])

mc_df = pd.DataFrame(rn.root2array(INFILE,treename='mctree'))
mc_df = mc_df.drop_duplicates(subset=['run','subrun','event'])

signal_mc_df = mc_df[mc_df.signal==1].copy().reindex()
back_mc_df   = mc_df[mc_df.signal==0].copy().reindex()

In [ ]:
# check if there is a good vertex or not
def correct(reco_vtx3d_,group_,chosen_idx_):

    ### use projected vertex
    
    # recox = reco_vtx3d_.vtx2d_x_v.values[chosen_idx_]
    # recoy = reco_vtx3d_.vtx2d_y_v.values[chosen_idx_]

    ### use circle vertex
    
    recox = reco_vtx3d_.circle_vtx_x_v.values[chosen_idx_]
    recoy = reco_vtx3d_.circle_vtx_y_v.values[chosen_idx_]
    
    mcx = group_.vtx2d_t.values[0]
    mcy = group_.vtx2d_w.values[0]
    
    dist=np.sqrt(np.power(recox-mcx,2)+np.power(recoy-mcy,2))

    dist.sort()

    dist0_b = False
    dist1_b = False
    
    if dist[0] <=7 : dist0_b = True
    if dist[1] <=7 : dist1_b = True
    
    return (dist0_b==True and dist1_b==True)
    

def do_selection(mc_sdf,reco_event_df,reco_vtx3d_df):

    selected = []

    mc_gdf  = mc_sdf.set_index(['run','subrun','event'])
    groups_ = mc_gdf.groupby(mc_gdf.index)
    
    for name, group in groups_:

        nvtx_b  = False
        close_b = False

        query_='run=={}&subrun=={}&event=={}'.format(name[0],name[1],name[2])
        
        reco_event = reco_event_df.query(query_)

        if reco_event.n_vtx3d.values > 0: 
            nvtx_b = True
        
        if nvtx_b == False: 
            continue
            
        reco_vtx3d = reco_vtx3d_df.query(query_)

        if ( reco_event.n_vtx3d.values != reco_vtx3d.vtx3d_id.size ):
            pass
            # print query_

        good_reco = False
        for idx_ in xrange(reco_event.n_vtx3d):

            if good_reco==True : break
            good_reco = correct(reco_vtx3d,group,idx_)

        if good_reco==True: 
            close_b=True

        if nvtx_b==True and close_b==True:
            selected.append(mc_sdf.query(query_).index.values[0])

    return np.array(selected)

# Reconstruction Efficiency on Signal

In [ ]:
mc_df=signal_mc_df
#mc_df=back_mc_df
n_reco_v = do_selection(mc_df,reco_event_df,reco_vtx3d_df)
print "Reco Efficiency is ", n_reco_v.size,"/",mc_df.index.size,"= ",float(n_reco_v.size)/float(mc_df.index.size)
print n_reco_v

In [ ]:
# Index by run,subrun,event
index_=['run','subrun','event']

#Signal dataframes
sig_event_df    = reco_event_df.copy()
sig_vtx3d_df    = reco_vtx3d_df.copy()
sig_particle_df = reco_particle_df.copy()

sig_mc_df       = signal_mc_df.set_index(index_)
sig_event_df    = sig_event_df.set_index(index_)
sig_vtx3d_df    = sig_vtx3d_df.set_index(index_)
sig_particle_df = sig_particle_df.set_index(index_)

sig_event_df    = sig_event_df.ix[sig_mc_df.index]
sig_vtx3d_df    = sig_vtx3d_df.ix[sig_mc_df.index]
sig_particle_df = sig_particle_df.ix[sig_mc_df.index]

#Background dataframes
bak_event_df    = reco_event_df.copy()
bak_vtx3d_df    = reco_vtx3d_df.copy()
bak_particle_df = reco_particle_df.copy()

bak_mc_df       = back_mc_df.set_index(index_)
bak_event_df    = bak_event_df.set_index(index_)
bak_vtx3d_df    = bak_vtx3d_df.set_index(index_)
bak_particle_df = bak_particle_df.set_index(index_)

bak_event_df    = bak_event_df.ix[sig_mc_df.index]
bak_vtx3d_df    = bak_vtx3d_df.ix[sig_mc_df.index]
bak_particle_df = bak_particle_df.ix[sig_mc_df.index]

In [ ]:
bins=np.arange(0,1000,100)
plt.hist(bak_mc_df.energyInit,bins=bins)
plt.hist(sig_mc_df.energyInit,bins=bins)
plt.show()